In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1337)

eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 50 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4

In [2]:
class scaledDotProduct(nn.Module):
    '''
        Attention(Q, K, V ) = softmax( QK^T/√d_k)V 
    
    '''
    #Takes number of embedded, head_size, context length
    def __init__(self, embn, hdim, con_l, drop=0.0):

        super(scaledDotProduct, self).__init__()
        #dim is (d_k) when sqrt'd it is meant to counter small gradients in large sets of queries and keys
        self.k = nn.Linear(embn, hdim, bias=False)
        self.q = nn.Linear(embn, hdim, bias=False)
        self.v = nn.Linear(embn, hdim, bias=False)
        self.d_k = np.sqrt(hdim)

        self.register_buffer('mask', torch.tril(torch.ones(con_l,con_l)))
        #Simple drop out 
        self.drop = nn.Dropout(drop)

    def forward(self, x, ret_att=False):
        #batch X length X dim
        B,T,C = x.shape
        k = self.k(x)
        q = self.q(x)

        n = torch.matmul(q, k.transpose(-2,-1)) * k.shape[-1]**-0.5 #BxTxhdim
        n = n.masked_fill(self.mask[:T,:T]==0, float('-inf'))
        #Drop out referenced later in paper but not in original diagram
        att = self.drop(F.softmax(n, dim=-1))

        v = self.v(x)

        out = torch.matmul(att, v)
        if ret_att:
            return out, att 
        return out
        
        


In [3]:
attention_test = scaledDotProduct(384,10,100)

v2 = torch.rand(1,8,384)
k2 = torch.rand(1,8,384)
q2 = torch.rand(1,8,384)

attention_test(v2)

tensor([[[ 7.3799e-02, -2.1080e-01, -1.6968e-01, -5.0506e-02,  3.5648e-01,
          -2.1507e-01, -1.4335e-02,  1.2158e-01,  1.4039e-01, -7.1718e-02],
         [-4.0966e-02, -2.2383e-01, -2.5205e-02,  1.3799e-01,  2.9074e-01,
          -2.2045e-01,  7.0719e-02,  1.3851e-01,  1.0924e-01,  2.6803e-02],
         [ 3.7568e-05, -1.2587e-01,  4.2005e-02,  2.8669e-02,  2.4530e-01,
          -2.6909e-01,  2.8913e-02,  8.6697e-02,  1.0704e-01,  3.7151e-02],
         [-9.0134e-02, -6.1785e-02,  7.6975e-02,  6.9112e-02,  2.9646e-01,
          -2.4842e-01,  3.2996e-02,  9.9801e-02,  5.6756e-02,  1.0289e-01],
         [-6.4113e-02, -7.6557e-02,  1.0644e-01,  3.3189e-02,  3.0053e-01,
          -2.1894e-01,  3.6116e-02,  1.0303e-01,  5.0443e-02,  1.3942e-01],
         [-4.6288e-02, -7.6528e-02,  1.1979e-01, -1.3275e-02,  2.3775e-01,
          -1.7745e-01,  6.0927e-02,  9.1160e-02,  8.1952e-02,  8.2527e-02],
         [-4.6428e-02, -1.4653e-01,  1.1806e-01,  3.3488e-02,  1.9384e-01,
          -1.8300e-

In [4]:
#Scaled dot product attention testing
#dim should be size of q and k

scaled_dot = scaledDotProduct(384,10,50,drop=0.2)


v = torch.rand(1,8,384)


print(scaled_dot(v))




tensor([[[ 5.2113e-01, -5.2166e-01, -1.5245e-01, -1.9464e-01,  1.2658e+00,
           4.3190e-01, -1.9973e-02,  6.7876e-02,  3.1785e-03, -7.0074e-02],
         [ 5.4967e-01, -4.3182e-01, -2.0202e-01, -2.1653e-01,  1.0719e+00,
           4.6660e-01,  1.9076e-02,  2.5814e-01,  1.6076e-01, -1.6141e-02],
         [ 3.6367e-01, -2.9071e-01, -1.3191e-01, -1.4278e-01,  7.2034e-01,
           3.0824e-01,  1.0922e-02,  1.6289e-01,  9.9700e-02, -1.3124e-02],
         [ 2.6756e-01, -1.5836e-01, -4.3613e-02, -8.8309e-02,  5.3854e-01,
           2.7328e-01, -1.2241e-01,  3.6059e-02, -3.2499e-02, -1.0748e-01],
         [ 3.6527e-01, -1.6424e-01, -2.0170e-01, -5.3736e-02,  6.7605e-01,
           3.9489e-01, -1.4605e-01,  1.7367e-01,  5.3122e-02,  1.1700e-04],
         [ 4.3739e-01, -1.5291e-01, -2.6700e-01, -6.6606e-02,  6.8358e-01,
           3.6720e-01, -1.8118e-01,  1.1687e-01,  2.8635e-02,  4.3078e-02],
         [ 4.5771e-01, -1.6209e-01, -1.9779e-01, -3.0877e-02,  7.2834e-01,
           3.2238e-

In [5]:
class multiHeadedAttention(nn.Module):
    def __init__(self, n_heads, dims, embn, con_l, dropout=0.0):
        super(multiHeadedAttention, self).__init__()
        #d_k=d_v = dims/h

        self.n_heads = n_heads

        self.attn = nn.ModuleList([scaledDotProduct(embn, dims, con_l) for _ in range(n_heads)])
        #Final linear layer after concat and attention
        self.fc = nn.Linear(n_heads*dims, embn)

        self.drop = nn.Dropout(dropout)
        

    def forward(self, x):
        out = torch.cat([h(x) for h in self.attn], dim=-1)
        out = self.drop(self.fc(out))
        return out

        


In [6]:
#heads, d_model, d_km d_v as per the paper
torch.manual_seed(1337)
multiHead = multiHeadedAttention(6, 50, 384, 512, dropout=0.2)

#batches, dims, dimensionalityxn_heads

v = torch.rand(1,50,384)


print(multiHead(v))


tensor([[[-0.1094,  0.3953, -0.0477,  ..., -0.0000,  0.0000, -0.0762],
         [-0.0667,  0.3789, -0.0155,  ..., -0.2896,  0.0174,  0.0112],
         [-0.0823,  0.0000,  0.0112,  ..., -0.2928, -0.0329,  0.0000],
         ...,
         [-0.0000,  0.2586,  0.0378,  ..., -0.2467, -0.0594,  0.0626],
         [-0.0000,  0.2572,  0.0000,  ..., -0.2471, -0.0629,  0.0634],
         [-0.0000,  0.0000,  0.0384,  ..., -0.2488, -0.0606,  0.0638]]],
       grad_fn=<MulBackward0>)


In [7]:
class positionFeedFoward(nn.Module):
    def __init__(self, inp, hid, drop=0.0):
        super(positionFeedFoward, self).__init__()
        self.w1 = nn.Linear(inp,4*hid)
        self.w2 = nn.Linear(4*hid,inp)
        self.drop = nn.Dropout(drop)

    def forward(self, x):

        x = self.w2(F.relu(self.w1(x)))
        x = self.drop(x)

        return x

In [8]:
class Decoder(nn.Module):
    '''Combinds MultiHeadedAttention and FeeForward, three layers'''
    def __init__(self, nheads, embn, con_l, drop=0.0):
        super(Decoder, self).__init__()
        head_size = embn // nheads
        self.slf_attn = multiHeadedAttention(nheads, head_size,embn, con_l, dropout=drop)
        
        self.ffn = positionFeedFoward(embn, embn, drop=drop)

        self.norm1 = nn.LayerNorm(embn)
        self.norm2 = nn.LayerNorm(embn)

    def forward(self, x):
        x = x + self.slf_attn(self.norm1(x))
        x = x + self.ffn(self.norm2(x))

        return x

In [9]:
#heads, d_model, d_km d_v as per the paper
enc = Decoder(8, 64, 512)
#batches, dims, dimensionalityxn_heads

v = torch.rand(1,512,64)


enc(v)

tensor([[[ 9.0589e-01,  1.7354e+00,  7.1903e-01,  ...,  1.2839e+00,
          -8.4718e-02,  6.1134e-04],
         [-1.4685e-01,  8.5962e-01, -1.6198e-01,  ...,  1.6304e-01,
           5.8517e-01, -4.6423e-01],
         [ 8.3383e-01,  1.4033e+00, -2.5442e-01,  ...,  3.2137e-01,
           1.1125e+00,  1.8170e-01],
         ...,
         [ 2.5373e-01,  5.0791e-01,  4.2961e-01,  ...,  4.7398e-01,
           9.7483e-01,  1.8666e-01],
         [ 6.0574e-01,  7.4977e-01,  2.4072e-01,  ...,  4.0424e-01,
           1.1312e+00,  4.6414e-02],
         [ 2.0085e-01,  1.2015e-01, -1.5217e-01,  ...,  5.4917e-01,
           7.0005e-01, -1.0259e-01]]], grad_fn=<AddBackward0>)

In [10]:
#Pytoch version adapted from here https://pub.aimind.so/creating-sinusoidal-positional-embedding-from-scratch-in-pytorch-98c49e153d6

class PosEncoding(nn.Module):
    def __init__(self, hid, n_pos=200):
        super(PosEncoding, self).__init__()

        self.register_buffer('pos_table', self._get_sinusoid_encoding_table(n_pos, hid))

    def _get_sinusoid_encoding_table(self, n_pos, hid):

        if hid %2 != 0:
            raise ValueError("Sinusoidal positional embedding cannot apply to odd token embedding dim={}".format(hid))
        
        positions = torch.arange(0,n_pos).unsqueeze_(1)
        embeds = torch.zeros(n_pos, hid)

        denom = torch.pow(10000, 2 * torch.arange(0, hid//2)/2)
        embeds[:, 0::2] = torch.sin(positions/denom)
        embeds[:, 1::2] = torch.cos(positions/denom)
        embeds = embeds.unsqueeze(0)

        return embeds
    
    def forward(self, x):
        return x + self.pos_table[:, :x.size(1)].clone().detach()




In [11]:
class languageModel(nn.Module):
    '''Decoder model'''
    def __init__(
            self, n_vocab, embn, n_layers, n_head, dropout=0.2 , con_l=200
    ):
        super(languageModel, self).__init__()
        self.con_l = con_l
        self.word_emb = nn.Embedding(n_vocab, embn)
        self.pos_enc = nn.Embedding(con_l, embn)
        self.stack = nn.Sequential(
            *[Decoder( n_head, embn, con_l, drop=dropout) for _ in range(n_layers)]
        )
       
        self.layer_norm = nn.LayerNorm(embn)
        self.fc = nn.Linear(embn, n_vocab)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x, tar=None):
        #batch, time
        B, T = x.shape

        tok = self.word_emb(x)
        pos = self.pos_enc(torch.arange(T, device=device))
        x = tok + pos
        x = self.stack(x)
        x = self.layer_norm(x)
        logits = self.fc(x)

        if tar is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            tar = tar.view(B*T)
            loss = F.cross_entropy(logits, tar)

        return logits, loss
    
    def generate(self, x, max_length):
        #x is a BxT array of in current context
        for _ in range(max_length):
            x_cond = x[:, -self.con_l:]
            logits, loss = self(x_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            x_next = torch.multinomial(probs, num_samples=1)
            x = torch.cat((x, x_next), dim=1)

        return x
    
    


In [12]:

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

torch.manual_seed(1337)

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits = model(X)[0]
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = Y.view(B*T)
            loss = F.cross_entropy(logits, targets)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [13]:
print(vocab_size)

65


In [12]:
model = languageModel(vocab_size,  384,6, 6, con_l=50
    )
m = model.to(device)
# print the number of parameters in the model
#print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
print(next(m.parameters()).is_cuda)
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    #B, T, C = logits.shape
    #logits = logits.view(B*T, C)
    #targets = yb.view(B*T)
    #loss = F.cross_entropy(logits, targets)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_length=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

True
step 0: train loss 4.2128, val loss 4.2117
step 500: train loss 1.7324, val loss 1.8882
step 1000: train loss 1.5333, val loss 1.7267
step 1500: train loss 1.4362, val loss 1.6588
step 2000: train loss 1.3867, val loss 1.6083
step 2500: train loss 1.3451, val loss 1.5763
step 3000: train loss 1.3136, val loss 1.5673
step 3500: train loss 1.2914, val loss 1.5662
step 4000: train loss 1.2631, val loss 1.5635
step 4500: train loss 1.2500, val loss 1.5503
step 4999: train loss 1.2263, val loss 1.5469

The king of war small Performeder; and Marcius Marcius,
And twenty, your joatings was misgest:
But, which is were all true, like deed ruge!

RIVERS:
And to be used-eated o' to common should Lady's at Lawful.

ROMEO:
This is there day, and Richard: I conquern us,
Thou call'dst good have no other:
He wise of that summost love, chose so but his choose
And begins of that say yon me, farewell.

MONTAGUE:
And if those gaves the meing hould, beseech you!

CLARENCE:
What no! Help for fourable, a

In [ ]:
print(decode(context[0].tolist()))
print("===========================")
print(decode(m.generate(context, max_length=500)[0].tolist()))

In [18]:
sinp = "wherefore art thou"
sinp = torch.tensor(encode(sinp)).unsqueeze(0).to(device)
print(decode(m.generate(sinp, max_length=500)[0].tolist()))

wherefore art thou, Norfolk, nor request;
If praying so good imposition in men, whispering
Fourth, raised by common imagined Duke of Hermirs;
But with salcS well drifick bones whereof another hate
Or endure a scooking bloody arms. What it subdeming,
Do impossible queen arage and linedly
Like trute us. O, this easing being of death!

GLOUCESTER:
What, good men do is banishment as him.

RICHARD:
Peace! I fight bawd abide twill nettle gatest, one
their place, but leavous now; strain braves or bed,
Which stalest blis
